# Fake Video Detection using eye blinking

## Importing liberaries

In [37]:
import numpy as np
import pandas as pd
import cv2
import dlib
from scipy.spatial import distance as dist
import os
notebook_path = os.path.abspath("Notebook.ipynb")

## Eye Blink and Eye Aspect Ratio calculations

### Function to calculate eye aspect ratio

In [2]:
def eye_aspect_ratio(eye):
    # compute the euclidean distance between the vertical eye landmarks
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])

    # compute the euclidean distance between the horizontal eye landmarks
    C = dist.euclidean(eye[0], eye[3])

    # compute the EAR
    ear = (A + B) / (2 * C)
    return ear

### Eye aspect ratio

In [87]:
EYE_AR_THRESH = 0.22
EYE_AR_CONSEC_FRAMES = 3
EAR_AVG = 0

### Counting Blinks in the video

In [90]:
# This function will create a OpenCV window
def count_blinks(video):
    # eye features regions
    RIGHT_EYE_POINTS = list(range(36, 42))
    LEFT_EYE_POINTS = list(range(42, 48))


    COUNTER = 0
    TOTAL = 0

    # to detect the facial region
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(os.path.join(os.path.dirname(notebook_path), "shape_predictor_68_face_landmarks.dat"
    ))
    # capture frame from local video
    # cap = cv2.VideoCapture("Home/Eye-detect-from-frame/download.jpeg")
    # capture video from live facecam
#     cap = cv2.VideoCapture(0)

    cap = cv2.VideoCapture(os.path.join(os.path.dirname(notebook_path), video))

    while True:
        # get the frame
        ret, frame = cap.read()
        #frame = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)
        if frame is not []:
            # convert the frame to grayscale
            try:
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            except:
                print(str(TOTAL)+" blinks found")
                cap.release()
                cv2.destroyAllWindows()
                return TOTAL
                break;
            # get the facial regions
            rects = detector(gray, 0)
            # loop through each of the facial regions
            for rect in rects:
                x = rect.left()
                y = rect.top()
                x1 = rect.right()
                y1 = rect.bottom()
                # get the facial landmarks
                landmarks = np.matrix([[p.x, p.y] for p in predictor(frame, rect).parts()])
                # get the left eye landmarks
                left_eye = landmarks[LEFT_EYE_POINTS]
                # get the right eye landmarks
                right_eye = landmarks[RIGHT_EYE_POINTS]
                # draw contours on the eyes
                left_eye_hull = cv2.convexHull(left_eye)
                right_eye_hull = cv2.convexHull(right_eye)
                cv2.drawContours(frame, [left_eye_hull], -1, (0, 255, 0), 1) # (image, [contour], all_contours, color, thickness)
                cv2.drawContours(frame, [right_eye_hull], -1, (0, 255, 0), 1)
                # compute the EAR for the left eye
                ear_left = eye_aspect_ratio(left_eye)
                # compute the EAR for the right eye
                ear_right = eye_aspect_ratio(right_eye)
                # compute the average EAR
                ear_avg = (ear_left + ear_right) / 2.0
                # detect the eye blink
                if ear_avg < EYE_AR_THRESH:
                    COUNTER += 1
                else:
                    if COUNTER >= EYE_AR_CONSEC_FRAMES:
                        TOTAL += 1

                    COUNTER = 0

                cv2.putText(frame, "Blinks {}".format(TOTAL), (10, 30), cv2.FONT_HERSHEY_DUPLEX, 0.7, (0, 255, 255), 1)
                cv2.putText(frame, "EAR {}".format(ear_avg), (10, 60), cv2.FONT_HERSHEY_DUPLEX, 0.7, (0, 255, 255), 1)
            cv2.imshow("Winks Found", frame)
            key = cv2.waitKey(1) & 0xFF
            # When key 'Q' is pressed, exit
            if key is ord('q'):
                print(str(TOTAL)+" blinks found")
                cap.release()
                cv2.destroyAllWindows()
                return TOTAL
                break
        else:
            cap.release()
            cv2.destroyAllWindows()
            print(str(TOTAL)+" blinks found")
            return TOTAL
    # release all resources
    cap.release()
    # destroy all windows
    cv2.destroyAllWindows()




In [111]:
# count_blinks("deepfake.mp4")

In [110]:
# arr

### Calculating EAR of each video

In [5]:
# This function will create a OpenCV window
def avg_ear(video):
    # eye features regions
    RIGHT_EYE_POINTS = list(range(36, 42))
    LEFT_EYE_POINTS = list(range(42, 48))


    COUNTER = 0
    TOTAL = 0
    EAR = 0
    n=0

    # to detect the facial region
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(os.path.join(os.path.dirname(notebook_path), "shape_predictor_68_face_landmarks.dat"
    ))
    # capture frame from local video
    # cap = cv2.VideoCapture("Home/Eye-detect-from-frame/download.jpeg")
    # capture video from live facecam
#     cap = cv2.VideoCapture(0)

    cap = cv2.VideoCapture(os.path.join(os.path.dirname(notebook_path), video))

    while True:
        # get the frame
        ret, frame = cap.read()
        #frame = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)
        if (frame is not []) & ret == True:
            # convert the frame to grayscale
            try:
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            except:
                print(str(EAR/n)+" is the average EAR of this video")
                cap.release()
                cv2.destroyAllWindows()
                return (EAR/n)
                break;
            # get the facial regions
            rects = detector(gray, 0)
            # loop through each of the facial regions
            for rect in rects:
                x = rect.left()
                y = rect.top()
                x1 = rect.right()
                y1 = rect.bottom()
                # get the facial landmarks
                landmarks = np.matrix([[p.x, p.y] for p in predictor(frame, rect).parts()])
                # get the left eye landmarks
                left_eye = landmarks[LEFT_EYE_POINTS]
                # get the right eye landmarks
                right_eye = landmarks[RIGHT_EYE_POINTS]
                # draw contours on the eyes
                left_eye_hull = cv2.convexHull(left_eye)
                right_eye_hull = cv2.convexHull(right_eye)
                cv2.drawContours(frame, [left_eye_hull], -1, (0, 255, 0), 1) # (image, [contour], all_contours, color, thickness)
                cv2.drawContours(frame, [right_eye_hull], -1, (0, 255, 0), 1)
                # compute the EAR for the left eye
                ear_left = eye_aspect_ratio(left_eye)
                # compute the EAR for the right eye
                ear_right = eye_aspect_ratio(right_eye)
                # compute the average EAR
                ear_avg = (ear_left + ear_right) / 2.0
                # detect the eye blink
                EAR+= ear_avg
                n+=1
                if ear_avg < EYE_AR_THRESH:
                    COUNTER += 1
                else:
                    if COUNTER >= EYE_AR_CONSEC_FRAMES:
                        TOTAL += 1

                    COUNTER = 0

                cv2.putText(frame, "Blinks {}".format(TOTAL), (10, 30), cv2.FONT_HERSHEY_DUPLEX, 0.7, (0, 255, 255), 1)
                cv2.putText(frame, "EAR {}".format(ear_avg), (10, 60), cv2.FONT_HERSHEY_DUPLEX, 0.7, (0, 255, 255), 1)
            cv2.imshow("Winks Found", frame)
            key = cv2.waitKey(1) & 0xFF
            # When key 'Q' is pressed, exit
            if key is ord('q'):
                print("Total no. of blinks are",TOTAL)
                break
        else:
            print(str(EAR/n)+" is the average EAR of this video")
            cap.release()
            cv2.destroyAllWindows()
            return (EAR/n)
            break
    # release all resources
    cap.release()
    # destroy all windows
    cv2.destroyAllWindows()

In [12]:
# avg_ear("1.mp4")

In [13]:
# count_blinks("1.mp4")

In [96]:
def count_blinks_ear(video):
    # eye features regions
    RIGHT_EYE_POINTS = list(range(36, 42))
    LEFT_EYE_POINTS = list(range(42, 48))


    COUNTER = 0
    TOTAL = 0
    EAR = 0
    n=0

    # to detect the facial region
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(os.path.join(os.path.dirname(notebook_path), "shape_predictor_68_face_landmarks.dat"
    ))
    cap = cv2.VideoCapture(os.path.join(os.path.dirname(notebook_path), video))
    frames = cap.get(cv2.CAP_PROP_FRAME_COUNT) 
    fps = int(cap.get(cv2.CAP_PROP_FPS)) 
    duration = int(frames / fps)
    while True:
        # get the frame
        ret, frame = cap.read()
        #frame = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)
        if frame is not []:
            # convert the frame to grayscale
            try:
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            except:
                print(str(TOTAL)+" blinks found")
                cap.release()
                cv2.destroyAllWindows()
                return (TOTAL,EAR/n,duration)
                break;
            # get the facial regions
            rects = detector(gray, 0)
            # loop through each of the facial regions
            for rect in rects:
                x = rect.left()
                y = rect.top()
                x1 = rect.right()
                y1 = rect.bottom()
                # get the facial landmarks
                landmarks = np.matrix([[p.x, p.y] for p in predictor(frame, rect).parts()])
                # get the left eye landmarks
                left_eye = landmarks[LEFT_EYE_POINTS]
                # get the right eye landmarks
                right_eye = landmarks[RIGHT_EYE_POINTS]
                # draw contours on the eyes
                left_eye_hull = cv2.convexHull(left_eye)
                right_eye_hull = cv2.convexHull(right_eye)
                cv2.drawContours(frame, [left_eye_hull], -1, (0, 255, 0), 1) # (image, [contour], all_contours, color, thickness)
                cv2.drawContours(frame, [right_eye_hull], -1, (0, 255, 0), 1)
                # compute the EAR for the left eye
                ear_left = eye_aspect_ratio(left_eye)
                # compute the EAR for the right eye
                ear_right = eye_aspect_ratio(right_eye)
                # compute the average EAR
                ear_avg = (ear_left + ear_right) / 2.0
                # detect the eye blink
                EAR+= ear_avg
                n+=1
                if ear_avg < EYE_AR_THRESH:
                    COUNTER += 1
                else:
                    if COUNTER >= EYE_AR_CONSEC_FRAMES:
                        TOTAL += 1

                    COUNTER = 0
        else:
            cap.release()
            cv2.destroyAllWindows()
            return (TOTAL,EAR/n,duration)
    # release all resources
    cap.release()
    # destroy all windows
    cv2.destroyAllWindows()

In [97]:
# count_blinks_ear("trump.mp4")

In [117]:
arr = os.listdir('./Celeb-DF/Celeb-real')

df = pd.DataFrame(columns = ['Name', 'Blinks','Average_EAR','FakeOReal','Duration']) 

for i,name in enumerate(arr):
#     if(i>=5):
#         break
    path = "./Celeb-DF/Celeb-real/"+str(name)
    a=count_blinks_ear(path)
    new_row = {'Name':name, 'Blinks':a[0],'Average_EAR':a[1],'FakeOReal':'Real','Duration':a[2]}
    df= df.append(new_row, ignore_index=True)

In [100]:
df

,Name,Blinks,Average_EAR,FakeOReal,Duration
0,id7_0001.mp4,4,0.296071,Real,9
1,id12_0006.mp4,0,0.336012,Real,10
2,id11_0007.mp4,3,0.334730,Real,10
3,id7_0007.mp4,4,0.305010,Real,13
4,id17_0005.mp4,10,0.254423,Real,10
...,...,...,...,...,...
153,id10_0007.mp4,5,0.334236,Real,16
154,id12_0002.mp4,0,0.357227,Real,12
155,id6_0000.mp4,2,0.336176,Real,21
156,id0_0002.mp4,13,0.197223,Real,11


In [101]:
df.to_csv('DF-real.csv', index = True)

In [109]:
count = 0
for i in df['Blinks']:
    if i == 1:
        count += 1 
count

9

In [118]:
arr = os.listdir('./Celeb-DF/Celeb-synthesis')

df2 = pd.DataFrame(columns = ['Name', 'Blinks','Average_EAR','FakeOReal','Duration']) 

for i,name in enumerate(arr):
    path = "./Celeb-DF/Celeb-synthesis/"+str(name)
    a=count_blinks_ear(path)
    new_row = {'Name':name, 'Blinks':a[0],'Average_EAR':a[1],'FakeOReal':'Fake','Duration':a[2]}
    df2= df2.append(new_row, ignore_index=True)

In [115]:
df2

,Name,Blinks,Average_EAR,FakeOReal,Duration
0,id1_id3_0004.mp4,12,0.262394,Fake,13
1,id3_id6_0008.mp4,1,0.294461,Fake,15
2,id4_id2_0009.mp4,8,0.300553,Fake,16
3,id9_id2_0007.mp4,9,0.261735,Fake,15
4,id13_id10_0000.mp4,1,0.380302,Fake,10
...,...,...,...,...,...
764,id3_id17_0005.mp4,11,0.244996,Fake,13
765,id16_id3_0007.mp4,4,0.295902,Fake,9
766,id2_id4_0003.mp4,7,0.262514,Fake,12
767,id2_id17_0006.mp4,9,0.283234,Fake,12


In [116]:
df2.to_csv('DF-synthesis.csv', index = True)